In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
select  app_id,feed_id,store_id,estimate,category_id ,device_id from plproxy.execute_select_nestloop(\$proxy\$ 
select  distinct app_id,feed_id,store_id,estimate,category_id ,device_id
    from aa.app_store_daily_estimate_1
    where 
      date = '2013-03-28'  order by estimate  asc limit 5 \$proxy\$) tbl 
      (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT, category_id INT, device_id SMALLINT ) order by estimate  asc limit 5 ;

EOF



In [0]:

import datetime
start = "2010-07-04"
end = "2020-04-09"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()

month_day=list()
for days in xrange(date_range.days):
    month_day.append(real_date1 + datetime.timedelta(days))


test_list= sorted(list(set([ d.strftime("%Y-%m-%d")[:7] for d in month_day ])))



In [0]:

from pyspark.sql import functions as F
# s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date=2020-03-28
def check_diff(month):
    df_1 = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}-*/".format(month))
    df_2 = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=weekly/date={}-*/".format(month))

    df_3 = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=weekly/date={}-*/".format(month))

    df_1.createOrReplaceTempView("daily_rank")
    df_2.createOrReplaceTempView("daily_est")
    df_3.createOrReplaceTempView("monthly")
    
    print "compare app id: " , month
    # spark.sql("select distinct app_id from monthly except all select distinct app_id from  daily ").show(2)
    # spark.sql("select distinct app_id from daily except all select distinct app_id from  monthly ").show(2)
    spark.sql('''select daily_est.app_id, daily_est.free_app_download, daily_est.country_code, daily_est.paid_app_download, daily_est.revenue, daily_est.device_code,daily_rank.category_id from daily_rank 
                join daily_est 
                on daily_rank.country_code= daily_est.country_code
                and daily_rank.device_code=daily_est.device_code
                and daily_rank.app_id = daily_est.app_id ''').createOrReplaceTempView("sum_category_daily")

    spark.sql(
        "select app_id, country_code, sum(free_app_download) as free_app_download , sum(paid_app_download) as paid_app_download , sum(revenue) as revenue, device_code ,category_id from sum_category_daily group by country_code,device_code,app_id, category_id ").createOrReplaceTempView(
        "sum_daily")

    spark.sql(
        '''select app_id, country_code, free_app_download, paid_app_download,  revenue, device_code, category_id, 
        RANKNUM() OVER (ORDER BY free_app_download) free_app_download,
        RANKNUM() OVER (ORDER BY paid_app_download) paid_app_download,
        RANKNUM() OVER (ORDER BY revenue) revenue,
         from sum_daily  ''').createOrReplaceTempView(
        "order_daily")


    # spark.sql("select app_id, country_code, free_app_download , paid_app_download , revenue, device_code,category_id from daily group by country_code,device_code,app_id,category_id ").createOrReplaceTempView("sum_daily")
    # spark.sql("select app_id,country_code,free_app_download,paid_app_download,revenue,device_code,category_id from sum_daily except all select app_id,country_code,free_app_download,paid_app_download,revenue,device_code,category_id from monthly").show(2)
    # spark.sql("select app_id,country_code,free_app_download,paid_app_download,revenue,device_code,category_id from monthly except all select app_id,country_code,free_app_download,paid_app_download,revenue,device_code,category_id from sum_daily").show(2)

    except_1 = spark.sql("select app_id,country_code,free_app_download,paid_app_download,revenue,device_code,category_id from sum_daily except all select app_id,country_code,free_app_download,paid_app_download,revenue,device_code,category_id from monthly").withColumn("date", F.lit(month) ).withColumn("type",F.lit("daily_monthly"))
    except_2 = spark.sql("select app_id,country_code,free_app_download,paid_app_download,revenue,device_code,category_id from monthly except all select app_id,country_code,free_app_download,paid_app_download,revenue,device_code ,category_id from sum_daily").withColumn("date", F.lit(month)).withColumn("type",F.lit("monthly_daily"))

    df_write_result = except_2.union(except_1)
    df_write_result.show()
    #if df_write_result.rdd.isEmpty():
    #    print 'pass'
#    else:
 #       print 'failed!!!!!!!' , month

    # from aadatapipelinecore.core.utils.retry import retry
    # def write_test_result(df_write_result):
    #     df_write_result.write.format("delta").save("s3://b2c-prod-data-pipeline-qa/aa.store/result_store_unified_monthly_category_count/",
    #                                       mode="append",
    #                                       partitionBy=["date"])
    # retry(write_test_result,(df_write_result,),{},interval=10)

sc.parallelize(map(check_diff, test_list), 1)

In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date=2020-01-01/").show()

In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=monthly/date=2020-01-31/").where("app_id=640073731 and country_code='MT' and device_code='ios-phone'").show()

In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=monthly/date=2020-01-31/device_code=android-all/ --human
